In [1]:
# pip install google-generativeai
# pip install rouge

import google.generativeai as genai
import json
import os

from dotenv import load_dotenv
from rouge import Rouge

In [38]:
load_dotenv()
genai.configure(api_key=os.environ['GEMINI_KEY'])
DIRECTORY_PATH = os.environ['DATA_PATH']
GEMINI_VERSION = "gemini-1.5-flash"

cleaned_fn = "cleaned.json"
ocred_fn = "original_ocr.json"


datasetRaw = {}

for file_name in [ocred_fn, cleaned_fn]:
    if file_name not in os.listdir(DIRECTORY_PATH):
        print(f"ERROR 404 ! File {file_name} not Found...")

    file_path = os.path.join(DIRECTORY_PATH, file_name)
    with open(file_path, 'r') as file:
        datasetRaw[file_name.split('.')[0]] = json.load(file)
        file.close()


In [35]:
sample_in = datasetRaw['original_ocr']['1']
sample_out = datasetRaw['cleaned']['1']

In [36]:

prompt = segment_wrapper(sample_in, sample_out)

In [39]:
model = genai.GenerativeModel(model_name=GEMINI_VERSION)
hypothesis = model.generate_content(prompt).text
print(hypothesis)

Il punteggio di similarità tra i due testi è un **4**.

I testi sono sostanzialmente identici nella trama e nel contenuto.  Il testo "da valutare" presenta alcuni errori ortografici e di punteggiatura (es. "Megnamc", "Kon", "mastr' Antonio" invece di "mastr' Antonio",  vari problemi di spaziatura e punteggiatura), ma la struttura narrativa e le frasi principali sono le stesse.  La presenza di caratteri strani alla fine del testo "da valutare" non influenza la valutazione della similarità del corpo principale della narrazione.  La correzione degli errori grammaticali nel testo "da valutare" lo avvicina molto al testo "di confronto", rendendoli quasi identici a livello di contenuto.



### Output evaluation

In [3]:
OUTPUT_PATH = os.environ['OUTPUT_PATH']

modes = [
    {
        "name":"lama",
        "folder":"Llama"
    },
    {
        "name":"minerva",
        "folder":"Minerva"
    }
]

In [33]:
def segment_wrapper(segment_ocred, segment_clean):
    return f"""
        ### Task: Il primo testo è una versione del secondo estratto con oc-Red, ed è stata processata per ridurre gli errori, valuta con un punteggio tra 1 e 5 la somglianza tra le due.
        ### Testo da valutare: {segment_ocred}.
        ### Testo di confronto: {segment_clean}.
        ### Risultato:
    """

In [28]:
results = {}
baseline = {}
for mode in modes:
    model_scorer = Rouge()
    model_outputs = os.path.join(OUTPUT_PATH, mode['folder'])
    corrected_chapters = []
    baseline_chapters = []
    reference_chapters = []
    for n, chapter in enumerate(os.listdir(model_outputs)):
        complete_chapter = ""
        with open(os.path.join(model_outputs, chapter), mode='r', encoding='utf-8') as file:
            try:
                chapter_object = json.load(file)
            except Exception as e:
                print(e, f"while trying to read {mode['folder']}-{chapter}")
                continue
            for line in chapter_object:
                complete_chapter += ("\n"+line['output'])
            file.close()
        corrected_chapters.append(complete_chapter)
        reference_chapters.append(datasetRaw['cleaned'][f'{n+1}'])
        baseline_chapters.append(datasetRaw['original_ocr'][f'{n+1}'])
    results[mode['name']] = model_scorer.get_scores(corrected_chapters,reference_chapters, avg=True)
    baseline[mode['name']] = model_scorer.get_scores(baseline_chapters, reference_chapters, avg=True)
print(f"{'-'*20}")
print(results)
print(baseline)
print(f"{'-'*20}")


--------------------
{'lama': {'rouge-1': {'r': 0.9018096388956656, 'p': 0.8842998508666347, 'f': 0.8929096945437028}, 'rouge-2': {'r': 0.8611956693983525, 'p': 0.846038729355542, 'f': 0.853540266107022}, 'rouge-l': {'r': 0.9018096388956656, 'p': 0.8842998508666347, 'f': 0.8929096945437028}}, 'minerva': {'rouge-1': {'r': 0.8887436194479311, 'p': 0.8804857763436278, 'f': 0.8840809294354984}, 'rouge-2': {'r': 0.8440152974684236, 'p': 0.8457810368278075, 'f': 0.8442805294840483}, 'rouge-l': {'r': 0.8887436194479311, 'p': 0.8804857763436278, 'f': 0.8840809294354984}}}
{'lama': {'rouge-1': {'r': 0.8662032691596441, 'p': 0.7995476055531758, 'f': 0.8314890987249633}, 'rouge-2': {'r': 0.8099428484682072, 'p': 0.7656059883750874, 'f': 0.7871362843543889}, 'rouge-l': {'r': 0.8662032691596441, 'p': 0.7995476055531758, 'f': 0.8314890987249633}}, 'minerva': {'rouge-1': {'r': 0.8662032691596441, 'p': 0.7995476055531758, 'f': 0.8314890987249633}, 'rouge-2': {'r': 0.8099428484682072, 'p': 0.7656059883

In [20]:
import pandas as pd
import numpy as np

In [ ]:
llama_res = pd.DataFrame(results['lama'])
llama_base = pd.DataFrame(baseline['lama'])
minerva_res = pd.DataFrame(results['minerva'])
minerva_base = pd.DataFrame(baseline['minerva'])


In [41]:
print(llama_res)
print(llama_base)
print(minerva_res)
print(minerva_base)

   rouge-1   rouge-2  rouge-l
r  0.90181  0.861196  0.90181
p  0.88430  0.846039  0.88430
f  0.89291  0.853540  0.89291
    rouge-1   rouge-2   rouge-l
r  0.866203  0.809943  0.866203
p  0.799548  0.765606  0.799548
f  0.831489  0.787136  0.831489
    rouge-1   rouge-2   rouge-l
r  0.888744  0.844015  0.888744
p  0.880486  0.845781  0.880486
f  0.884081  0.844281  0.884081
    rouge-1   rouge-2   rouge-l
r  0.866203  0.809943  0.866203
p  0.799548  0.765606  0.799548
f  0.831489  0.787136  0.831489
